### Tree-Based Methods

In this chapter, we describe tree-based methods for regression and classifi
cation. These involve stratifying or segmenting the predictor space into a
number of simple regions. In order to make a prediction for a given ob
servation, we typically use the mean or the mode response value for the
training observations in the region to which it belongs. Since the set of
splitting rules used to segment the predictor space can be summarized in
a tree, these types of approaches are known as decision tree methods.
Tree-based methods are simple and useful for interpretation. However,
they typically are not competitive with the best supervised learning ap
proaches, such as those seen in Chapters 6 and 7, in terms of prediction
accuracy. Hence in this chapter we also introduce bagging, random forests,
boosting, and Bayesian additive regression trees. Each of these approaches
involves producing multiple trees which are then combined to yield a single
consensus prediction. We will see that combining a large number of trees
can often result in dramatic improvements in prediction accuracy, at the
expense of some loss in interpretation.

#### The Basics of Decision Trees

Decision trees can be applied to both regression and classification problems.
We first consider regression problems, and then move on to classification.

##### Regression Trees

In order to motivate regression trees, we begin with a simple example.

Predicting Baseball Players’ Salaries Using Regression Trees

We use the Hitters data set to predict a baseball player’s Salary based on
Years (the number of years that he has played in the major leagues) and
Hits (the number of hits that he made in the previous year). We first remove
observations that are missing Salary values, and log-transform Salary so
that its distribution has more of a typical bell-shape. (Recall that Salary
is measured in thousands of dollars.)
Figure 8.1 shows a regression tree fit to this data. It consists of a series
of splitting rules, starting at the top of the tree. The top split assigns
observations having Years<4.5 to the left branch.1 The predicted salary
for these players is given by the mean response value for the players in
the data set with Years<4.5. For such players, the mean log salary is 5.107,
and so we make a prediction of e5.107 thousands of dollars, i.e. $165,174, for
these players. Players with Years>=4.5 are assigned to the right branch, and
then that group is further subdivided by Hits. Overall, the tree stratifies
or segments the players into three regions of predictor space: players who
have played for four or fewer years, players who have played for five or more
years and who made fewer than 118 hits last year, and players who have
played for five or more years and who made at least 118 hits last year. These
three regions can be written as R1 ={X | Years<4.5}, R2 ={X | Years>=4.5,
Hits<117.5}, and R3 ={X | Years>=4.5, Hits>=117.5}. Figure 8.2 illustrates the regions as a function of Years and Hits. The predicted salaries for these
three groups are $1,000 e5.107 =$165,174, $1,000 e5.999 =$402,834, and
$1,000 e6.740 =$845,346 respectively.
In keeping with the tree analogy, the regions R1, R2, and R3 are known as
terminal nodes or leaves of the tree. As is the case for Figure 8.1, decision terminal
trees are typically drawn upside down, in the sense that the leaves are at
the bottom of the tree. The points along the tree where the predictor space
is split are referred to as internal nodes. In Figure 8.1, the two internal internal
nodes are indicated by the text Years<4.5 and Hits<117.5. We refer to the
segments of the trees that connect the nodes as branches.
We might interpret the regression tree displayed in Figure 8.1 as follows:
Years is the most important factor in determining Salary, and players with
less experience earn lower salaries than more experienced players. Given
that a player is less experienced, the number of hits that he made in the
previous year seems to play little role in his salary. But among players who
have been in the major leagues for five or more years, the number of hits
made in the previous year does affect salary, and players who made more
hits last year tend to have higher salaries. The regression tree shown in
Figure 8.1 is likely an over-simplification of the true relationship between
Hits, Years, and Salary. However, it has advantages over other types of
regression models (such as those seen in Chapters 3 and 6): it is easier to
interpret, and has a nice graphical representation.
Prediction via Stratification of the Feature Space
Wenowdiscuss the process of building a regression tree. Roughly speaking,
there are two steps.
1. We divide the predictor space — that is, the set of possible values
for X1,X2,...,Xp — into J distinct and non-overlapping regions,
R1,R2,...,RJ.
2. For every observation that falls into the region Rj, we make the same
prediction, which is simply the mean of the response values for the
training observations in Rj.
For instance, suppose that in Step 1 we obtain two regions, R1 and R2,
and that the response mean of the training observations in the first region
is 10, while the response mean of the training observations in the second
region is 20. Then for a given observation X = x, if x R1 we will predict
a value of 10, and if x R2 we will predict a value of 20.
We now elaborate on Step 1 above. How do we construct the regions
R1,...,RJ? In theory, the regions could have any shape. However, we
choose to divide the predictor space into high-dimensional rectangles, or
boxes, for simplicity and for ease of interpretation of the resulting predic
tive model. The goal is to find boxes R1,...,RJ that minimize the RSS,
given by For instance, suppose that in Step 1 we obtain two regions, $ R_1 $ and $ R_2 $, and that the response mean of the training observations in the first region is 10, while the response mean of the training observations in the second region is 20. Then for a given observation $ X = x $, if $ x \in R_1 $ we will predict a value of 10, and if $ x \in R_2 $ we will predict a value of 20.

We now elaborate on Step 1 above. How do we construct the regions $ R_1, \ldots, R_J $? In theory, the regions could have any shape. However, we choose to divide the predictor space into high-dimensional rectangles, or boxes, for simplicity and for ease of interpretation of the resulting predictive model. The goal is to find boxes $ R_1, \ldots, R_J $ that minimize the RSS, given by

$$
\sum_{j=1}^{J} \sum_{i \in R_j} (y_i - \hat{y}_{R_j})^2,
$$

where $ \hat{y}_{R_j} $ is the mean response for the training observations within the $ j $th box. Unfortunately, it is computationally infeasible to consider every possible partition of the feature space into $ J $ boxes. For this reason, we take a top-down, greedy approach that is known as recursive binary splitting. The approach is top-down because it begins at the top of the tree (at which point all observations belong to a single region) and then successively splits the predictor space; each split is indicated via two new branches further down on the tree. It is greedy because at each step of the tree-building process, the best split is made at that particular step, rather than looking ahead and picking a split that will lead to a better tree in some future step.

In order to perform recursive binary splitting, we first select the predictor $ X_j $ and the cutpoint $ s $ such that splitting the predictor space into the regions $ \{X | X_j < s\} $ and $ \{X | X_j \geq s\} $ leads to the greatest possible reduction in RSS. (The notation $ \{X | X_j < s\} $ means the region of predictor space in which $ X_j $ takes on a value less than $ s $.) That is, we consider all predictors $ X_1, \ldots, X_p $, and all possible values of the cutpoint $ s $ for each of the predictors, and then choose the predictor and cutpoint such that the resulting tree has the lowest RSS. In greater detail, for any $ j $ and $ s $, we define the pair of half-planes

$$
R_1(j,s) = \{X | X_j < s\} \quad \text{and} \quad R_2(j,s) = \{X | X_j \geq s\},
$$

and we seek the value of $ j $ and $ s $ that minimize the equation

$$
\sum_{i: x_i \in R_1(j,s)} (y_i - \hat{y}_{R_1})^2 + \sum_{i: x_i \in R_2(j,s)} (y_i - \hat{y}_{R_2})^2,
$$

where $ \hat{y}_{R_1} $ is the mean response for the training observations in $ R_1(j,s) $, and $ \hat{y}_{R_2} $ is the mean response for the training observations in $ R_2(j,s) $. Finding the values of $ j $ and $ s $ that minimize the above expression can be done quite quickly, especially when the number of features $ p $ is not too large.

Next, we repeat the process, looking for the best predictor and best cutpoint in order to split the data further so as to minimize the RSS within each of the resulting regions. However, this time, instead of splitting the
entire predictor space, we split one of the two previously identified regions.
Wenowhavethree regions. Again, we look to split one of these three regions
further, so as to minimize the RSS. The process continues until a stopping
criterion is reached; for instance, we may continue until no region contains
more than five observations.
Once the regions R1,...,RJ have been created, we predict the response
for a given test observation using the mean of the training observations in
the region to which that test observation belongs.
A five-region example of this approach is shown in Figure 8.3.


Tree Pruning


The process described above may produce good predictions on the training
set, but is likely to overfit the data, leading to poor test set performance.
This is because the resulting tree might be too complex. A smaller tree Variance and better interpretation at the cost of a little bias. One possible alternative to the process described above is to build the tree only so long as the decrease in the RSS due to each split exceeds some (high) threshold. This strategy will result in smaller trees, but is too short-sighted since a seemingly worthless split early on in the tree might be followed by a very good split—that is, a split that leads to a large reduction in RSS later on. Therefore, a better strategy is to grow a very large tree $ T_0 $, and then prune it back in order to obtain a subtree. How do we determine the best way to prune the tree? Intuitively, our goal is to select a subtree that leads to the lowest test error rate. Given a subtree, we can estimate its test error using cross-validation or the validation set approach. However, estimating the cross-validation error for every possible subtree would be too cumbersome, since there is an extremely large number of possible subtrees. Instead, we need a way to select a small set of subtrees for consideration.

Cost complexity pruning—also known as weakest link pruning—gives us a way to do just this. Rather than considering every possible subtree, we consider a sequence of trees indexed by a nonnegative tuning parameter $ \alpha $. For each value of $ \alpha $ there corresponds a subtree $ T \subset T_0 $ such that 

$$
\sum_{m=1}^{|T|} \sum_{i: x_i \in R_m} (y_i - \hat{y}_{R_m})^2 + |\mathcal{T}|\alpha
$$

is as small as possible. Here $ |T| $ indicates the number of terminal nodes of the tree $ T $, $ R_m $ is the rectangle (i.e. the subset of predictor space) corresponding to the $ m $th terminal node, and $ \hat{y}_{R_m} $ is the predicted response associated with $ R_m $—that is, the mean of the training observations in $ R_m $.

The tuning parameter controls a trade-off between the subtree’s complexity and its fit to the training data. When $ \alpha = 0 $, then the subtree $ T $ will simply equal $ T_0 $, because then the above equation just measures the training error. However, as $ \alpha $ increases, there is a price to pay for having a tree with many terminal nodes, and so the quantity will tend to be minimized for a smaller subtree. Equation (8.4) is reminiscent of the lasso (6.7) from Chapter 6, in which a similar formulation was used in order to control the complexity of a linear model.

It turns out that as we increase $ \alpha $ from zero in (8.4), branches get pruned from the tree in a nested and predictable fashion, so obtaining the whole sequence of subtrees as a function of $ \alpha $ is easy. We can select a value of $ \alpha $ using a validation set or using cross-validation. We then return to the full data set and obtain the subtree corresponding to $ \alpha $. This process is summarized in Algorithm 8.1.

Figures 8.4 and 8.5 display the results of fitting and pruning a regression tree on the Hitters data, using nine of the features. First, we randomly divided the data set in half, yielding 132 observations in the training set and 131 observations in the test set. We then built a large regression tree on the training data and varied $ \alpha $ in (8.4) in order to create subtrees with different numbers of terminal nodes. Finally, we performed six-fold cross validation in order to estimate the cross-validated MSE of the trees as a function of $\alpha$. (We chose to perform six-fold cross-validation because 132 is an exact multiple of six.) The unpruned regression tree is shown in Figure 8.4. The green curve in Figure 8.5 shows the CV error as a function of the number of leaves, while the orange curve indicates the test error. Also shown are standard error bars around the estimated errors. For reference, the training error curve is shown in black. The CV error is a reasonable approximation of the test error: the CV error takes on its minimum for a three-node tree, while the test error also dips down at the three-node tree (though it takes on its lowest value at the ten-node tree). The pruned tree containing three terminal nodes is shown in Figure 8.1.



**Algorithm 8.1 $Building$ $a$ $Regression$ $Tree$**

1. Use recursive binary splitting to grow a large tree on the training data, stopping only when each terminal node has fewer than some minimum number of observations.
2. Apply cost complexity pruning to the large tree in order to obtain a sequence of best subtrees, as a function of $ \alpha $.
3. Use K-fold cross-validation to choose $ \alpha $. That is, divide the training observations into K folds. For each $ k = 1, \ldots, K $:
   (a) Repeat Steps 1 and 2 on all but the kth fold of the training data.
   (b) Evaluate the mean squared prediction error on the data in the left-out kth fold, as a function of $ \alpha $. Average the results for each value of $ \alpha $, and pick $ \alpha $ to minimize the average error.
4. Return the subtree from Step 2 that corresponds to the chosen value of $ \alpha $.





##### Classification Trees



A classification tree is very similar to a regression tree, except that it is used to predict a qualitative response rather than a quantitative one. Recall that for a regression tree, the predicted response for an observation is given by the mean response of the training observations that belong to the same terminal node. In contrast, for a classification tree, we predict that each observation belongs to the most commonly occurring class of training observations in the region to which it belongs. In interpreting the results of a classification tree, we are often interested not only in the class prediction corresponding to a particular terminal node region, but also in the class proportions among the training observations that fall into that region.

The task of growing a classification tree is quite similar to the task of growing a regression tree. Just as in the regression setting, we use recursive Binary splitting to grow a classification tree. However, in the classification setting, RSS cannot be used as a criterion for making the binary splits. A natural alternative to RSS is the classification error rate. Since we plan to assign an observation in a given region to the most commonly occurring class of training observations in that region, the classification error rate is simply the fraction of the training observations in that region that do not belong to the most common class:

$$
E = 1 - \max_k (\hat{p}_{mk}).
$$

Here $ \hat{p}_{mk} $ represents the proportion of training observations in the mth region that are from the kth class. However, it turns out that classification error is not sufficiently sensitive for tree-growing, and in practice two other measures are preferable.

The Gini index is defined by

$$
G = \sum_{k=1}^{K} \hat{p}_{mk}(1 - \hat{p}_{mk}),
$$

a measure of total variance across the K classes. It is not hard to see that the Gini index takes on a small value if all of the $ \hat{p}_{mk} $s are close to zero or one. For this reason, the Gini index is referred to as a measure of Node purity—a small value indicates that a node contains predominantly observations from a single class.

An alternative to the Gini index is entropy, given by

$$
D = -\sum_{k=1}^{K} \hat{p}_{mk} \log \hat{p}_{mk}.
$$

Since $0 \leq \hat{p}_{mk} \leq 1$, it follows that $0 \leq -\hat{p}_{mk} \log \hat{p}_{mk}$. One can show that the entropy will take on a value near zero if the $\hat{p}_{mk}$s are all near zero or near one. Therefore, like the Gini index, the entropy will take on a small value if the mth node is pure. In fact, it turns out that the Gini index and the entropy are quite similar numerically.

When building a classification tree, either the Gini index or the entropy are typically used to evaluate the quality of a particular split, since these two approaches are more sensitive to node purity than is the classification error rate. Any of these three approaches might be used when pruning the tree, but the classification error rate is preferable if prediction accuracy of the final pruned tree is the goal.

Figure 8.6 shows an example on the Heart data set. These data contain a binary outcome HD for 303 patients who presented with chest pain. An outcome value of Yes indicates the presence of heart disease based on an angiographic test, while No means no heart disease. There are 13 predictors including Age, Sex, Chol (a cholesterol measurement), and other heart and lung function measurements. Cross-validation results in a tree with six terminal nodes.

In our discussion thus far, we have assumed that the predictor variables take on continuous values. However, decision trees can be constructed even in the presence of qualitative predictor variables. For instance, in the Heart data, some of the predictors, such as Sex, Thal (Thallium stress test), and ChestPain, are qualitative. Therefore, a split on one of these variables
amounts to assigning some of the qualitative values to one branch and
assigning the remaining to the other branch. In Figure 8.6, some of the in
ternal nodes correspond to splitting qualitative variables. For instance, the
top internal node corresponds to splitting Thal. The text Thal:a indicates
that the left-hand branch coming out of that node consists of observations
with the first value of the Thal variable (normal), and the right-hand node
consists of the remaining observations (fixed or reversible defects). The text
ChestPain:bc two splits down the tree on the left indicates that the left-hand
branch coming out of that node consists of observations with the second
and third values of the ChestPain variable, where the possible values are
typical angina, atypical angina, non-anginal pain, and asymptomatic.
Figure 8.6 has a surprising characteristic: some of the splits yield two
terminal nodes that have the same predicted value. For instance, consider
the split RestECG<1 near the bottom right of the unpruned tree. Regardless
of the value of RestECG, a response value of Yes is predicted for those observations. Why, then, is the split performed at all? The split is performed because it leads to increased node purity. That is, all 9 of the observations corresponding to the right-hand leaf have a response value of Yes, whereas 7/11 of those corresponding to the left-hand leaf have a response value of Yes. Why is node purity important? Suppose that we have a test observation that belongs to the region given by that right-hand leaf. Then we can be pretty certain that its response value is Yes. In contrast, if a test observation belongs to the region given by the left-hand leaf, then its response value is probably Yes, but we are much less certain. Even though the split RestECG < 1 does not reduce the classification error, it improves the Gini index and the entropy, which are more sensitive to node purity.



##### Trees Versus Linear Models



Regression and classification trees have a very different flavor from the more classical approaches for regression and classification presented in Chapters 3 and 4. In particular, linear regression assumes a model of the form

$$
f(X) = \beta_0 + \sum_{j=1}^{p} X_j \beta_j,
$$

whereas regression trees assume a model of the form

$$
f(X) = \sum_{m=1}^{M} c_m \cdot \mathbb{1}(X \in R_m)
$$

where $ R_1, \ldots, R_M $ represent a partition of feature space, as in Figure 8.3. Which model is better? It depends on the problem at hand. If the relationship between the features and the response is well approximated by a linear model as in (8.8), then an approach such as linear regression will likely work well, and will outperform a method such as a regression tree that does not exploit this linear structure. If instead there is a highly non-linear and complex relationship between the features and the response as indicated by model (8.9), then decision trees may outperform classical approaches. An illustrative example is displayed in Figure 8.7. The relative performances of tree-based and classical approaches can be assessed by estimating the test error, using either cross-validation or the validation set approach (Chapter 5).

Of course, other considerations beyond simply test error may come into play in selecting a statistical learning method; for instance, in certain settings, prediction using a tree may be preferred for the sake of interpretability and visualization.


##### Advantages and Disadvantages of Trees

Decision trees for regression and classification have a number of advantages over the more classical approaches seen in Chapters 3 and 4:

- Trees are very easy to explain to people. In fact, they are even easier to explain than linear regression!